# Metrics
In this notebook we will calculate and plot the metrics coming from the YOLO training runs.

# Load Data
As a first step, let's fetch the results from our training run.

In [78]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

In [79]:
!curl -L https://aml-2023.s3.eu-north-1.amazonaws.com/final-project/yolo_runs_epoch_90.zip > yolo_runs_epoch_90.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 20.8M    0 88891    0     0   109k      0  0:03:15 --:--:--  0:03:15  109k
  2 20.8M    2  494k    0     0   279k      0  0:01:16  0:00:01  0:01:15  279k
  4 20.8M    4 1055k    0     0   380k      0  0:00:56  0:00:02  0:00:54  380k
  7 20.8M    7 1497k    0     0   395k      0  0:00:53  0:00:03  0:00:50  396k
  8 20.8M    8 1871k    0     0   391k      0  0:00:54  0:00:04  0:00:50  392k
 10 20.8M   10 2245k    0     0   389k      0  0:00:54  0:00:05  0:00:49  433k
 11 20.8M   11 2466k    0     0   363k      0  0:00:58  0:00:06  0:00:52  393k
 12 20.8M   12 2653k    0     0   340k      0  0:01:02  0:00:07  0:00:55  317k
 13 20.8M   13 2874k    0     0   327k      0  0:01:05  0:00:08  0:00:57  275k
 14 20.8M   14 3010k    0     0   306k      0  0:01

And extract into a chosen directory.

In [80]:
import zipfile

run_data_dir = "run_data"
Path(run_data_dir).mkdir(exist_ok=True, parents=True)

with zipfile.ZipFile("yolo_runs_epoch_90.zip", 'r') as zip_ref:
    zip_ref.extractall(run_data_dir)

Let's load the run dataframe.

In [ ]:
run_results = pd.read_csv("data/yolo_runs_epoch_90/runs/detect/train/results.csv")

Then, let's fetch the training and validation data. This we need for the validation of the YOLO model at the end.

In [ ]:
!fetch_data.sh --type yolo --output garbage_sub --percentage subset

# Extract Train and Validation Results
Then, we extract the training and validation columns from the dataframe.

In [41]:
train_columns = list(filter(lambda col_name: "train" in col_name, run_results.columns))
train_results = run_results[train_columns]

val_columns = list(filter(lambda col_name: "val" in col_name, run_results.columns))
val_results = run_results[val_columns]

Create the output directory if it doesn't already exist.

In [42]:
Path("metrics_plots").mkdir(exist_ok=True, parents=True)

# Plot
Create a function to plot the loss and save it if requested. We always plot the training and validation loss values for a specific type of loss together, e.g. box loss.

In [43]:
import os

def plot_loss(train, val, name, save=False, save_dir=None, save_format="svg"):
    """Creates a figure, axis tuple for the train and validation losses.

    To save the plot, both the save and save_dir parameters must be set. Images will be saved as svg by default.

    :arg
        train - an iterable of the training loss.
        val - an iterable of the validation loss.
        name - the name of the loss, e.g. box loss. This will be the title of the plot.
        save - whether to save the plot.
        save_dir - the directory where to save the plot, must exist before calling this function.
        save_format - the format to save the plot in, default is svg.

    :return
        fig, ax - the figure and axis object of the plot.
    """
    fig, ax = plt.subplots()

    ax.plot(train, label="Train")
    ax.plot(val, label="Val")
    ax.set_title(name)
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.grid(True)
    ax.legend()

    if save and save_dir:
        file_name = f"{name.replace(' ', '_')}.{save_format}"
        fig.savefig(os.path.join(save_dir, file_name), format=save_format)
    return fig, ax

Finally, we call the `plot_loss` method to create and save the loss plots in the specified directory.

In [44]:
plot_loss(train_results.iloc[:, 0], val_results.iloc[:, 0], "Box Loss", save=True, save_dir="metrics_plots")
plot_loss(train_results.iloc[:, 1], val_results.iloc[:, 1], "Classification Loss", save=True, save_dir="metrics_plots")
plot_loss(train_results.iloc[:, 2], val_results.iloc[:, 2], "Distributional Focal Loss", save=True, save_dir="metrics_plots")

(<Figure size 640x480 with 1 Axes>,
 <Axes: title={'center': 'Distributional Focal Loss'}, xlabel='Epoch', ylabel='Loss'>)

# Classification Metrics
Next, let's do the classification metrics:

* precision
* precision-recall
* F1

In [53]:
from ultralytics import YOLO
from ultralytics.nn.tasks import DetectionModel
import torch

# Setup Model
As a first step, we need to setup up the model by doing the following:

1. Create a `DetectionModel` with the garbage architecture, basically just use a single class instead of the many that are normally used.
2. Load the best weights from the training into this model.
3. Create the YOLO model with the same best weights and with a detection task, since we want to do object detection here.
4. Assign the detection model to the `model` field of the YOLO object. This is a bit hacky but it's the only way we can let YOLO know that it should only predict a single class.

In [54]:
det = DetectionModel("model.yaml")
det.load(torch.load("data/yolo_runs_epoch_90/runs/detect/train/weights/best.pt"))
model = YOLO(model="data/yolo_runs_epoch_90/runs/detect/train/weights/best.pt", task="detect")  # load a pretrained model (recommended for training)
model.model = det


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

# Model Validation
Next, we validate the model on the **test** data by simply calling the `val` method with the path to the `.yaml` file where we specify the dataset. This will return a `Metrics` object from which we can access all the metrics we are interested in.

In [70]:
# Validate the model
data_path = os.path.abspath("garbage_sub/data.yaml")
metrics = model.val(data=data_path)  # no arguments needed, dataset and settings remembered

Ultralytics YOLOv8.0.225 🚀 Python-3.9.0 torch-2.1.1+cpu CPU (Intel Core(TM) i5-6200U 2.30GHz)


val: Scanning C:\Users\Jonas\Desktop\Uni\MSc\Year-2\AML\Assignments\final-project\final-project\garbage_sub\valid\labels.cache... 28 images, 1 backgrounds, 0 corrupt: 100%|██████████| 28/28 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:11<00:00,  5.88s/it]


                   all         28         31      0.899      0.577      0.713       0.42
Speed: 7.9ms preprocess, 389.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to C:\Users\Jonas\Desktop\Uni\MSc\Year-2\AML\Assignments\final-project\final-project\runs\detect\val4


# Plotting the Metrics
Now, let's plot the metrics. First, let's create a plot function that we can reuse, similar to the `plot_loss` function above.

In [73]:
def plot_metric(data, xlabel, ylabel, title, save=False, save_dir=None, save_format="svg"):
    """Plots a metric of the YOLOv8 model.

    arg:
        data - the data to plot. Should just be a single variable (statistically speaking).
        xlabel - the label of the x axis.
        ylabel - the label of the y axis.
        title - the title of the plot.
        save - whether to save the plot.
        save_dir - the directory where to save the plot, must exist before calling this function.
        save_format - the format to save the plot in, default is svg.

    :return
        fig, ax - the figure and axis object of the plot.
    """

    fig, ax = plt.subplots()

    ax.plot(data)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)

    ax.grid(True)

    if save and save_dir:
        file_name = f"{title.replace(' ', '_')}.{save_format}"
        fig.savefig(os.path.join(save_dir, file_name), format=save_format)
    return fig, ax

# Precision
Precision is a measurement of how many of our predicted true positives are actually true positives:

$$
precision = \frac{TP}{TP + FP}
$$

In [74]:
plot_metric(metrics.box.p_curve.T, "Confidence", "Precision", "Precision-Confidence")

(<Figure size 640x480 with 1 Axes>,
 <Axes: title={'center': 'Precision-Confidence'}, xlabel='Confidence', ylabel='Precision'>)